In [1]:
from databricks.feature_store import *

from pyspark.sql.types import *
from pyspark.sql.functions import when, lit, substring, col, coalesce, lead, datediff, expr, posexplode, year, concat, lpad, month, last_day, add_months, trunc, date_add, greatest
from pyspark.sql.window import Window

ModuleNotFoundError: No module named 'databricks'

In [0]:
fs = FeatureStoreClient()

ft_policy = fs.read_table(name='ess_feature_store.policy')
ft_policy = (ft_policy.filter(col('PRODUCT_IND')=='Rubix')
             .select(
               'POLICYID',
               'POLICY_EFF_DT',
               'POLICY_EXP_DT',
               'POLICYSTATUS',             
               'POLICYISSUETYPE',
               'COMPANY',
               'POLICYSTATE',
               'STATE_ADMITTED')
            )

ft_vehicle_policy = fs.read_table(name='ess_feature_store.vehicle_policy')
ft_vehicle_policy = ft_vehicle_policy.select('POLICYID',
                                            'POLICY_EFF_DT',
                                            'NEW_VENTURE',
                                            'PKGCOMBO',
                                            'YRSINBUSCT',
                                            'YRSOFINSCT',
                                            'POLICY_SEGMENT',
                                            'OPERATION_EXPOSURE_POLICY',
                                            'ZONE_GRPCD_POLICY',
                                            'DOT_NUMBER',
                                            'SCHEDULED_FLEET')

ft_vehicle_exposure = fs.read_table(name='ess_feature_store.vehicle_exposure')
ft_vehicle_exposure = (ft_vehicle_exposure.select('POLICYID',
                                                'POLICY_EFF_DT',
                                                'VEHICLE_NO', 
                                                'VEHICLE_SEGMENT',
                                                'VEHICLE_TERRITORY',
                                                'VEHICLE_STATE',
                                                'MODEL_YEAR', 
                                                'EQUIPMENT_AGE',
                                                'VIN', 
                                                'RADIUS_GROUP',
                                                'CLASS_CD',
                                                'ZONE_GRPCD_VEHICLE',
                                                'VEHICLE_ZIPCD',
                                                'CLASS_PRIMARY',
                                                'CLASS_SECONDARY',
                                                'VEHICLE_TYPE',
                                                'TRANSFORM_TERRITORY',
                                                'TERRITORY_GRP',
                                                'OPERATION_EXPOSURE_VEHICLE',
                                                'EARNED_EXP_TRANS',
                                                'EARNED_EXPOSURES')
                       .where(col('EARNED_EXPOSURES').isNotNull())  # Null values are non power units
                       .where(~((col('EARNED_EXPOSURES')==0) & (col('EARNED_EXP_TRANS')==0)))
                       )

ft_vehicle_pricing = fs.read_table(name='ess_feature_store.vehicle_pricing')
ft_vehicle_pricing = ft_vehicle_pricing.select('POLICYID',
                                              'POLICY_EFF_DT',
                                              'AVG_DRIVER_SURCHARGE',
                                              'LIAB_SCHMOD',
                                              'LIAB_EXPERIMOD')

ft_vehicle_rating = fs.read_table(name='ess_feature_store.vehicle_rating')
ft_vehicle_rating = ft_vehicle_rating.select('POLICYID',
                                            'POLICY_EFF_DT',
                                            'VEHICLE_NO',
                                            'RERATED_PREMIUM',
                                            'BASE_RATE',
                                            'RELATIVITY_EQUIPMENT',
                                            'RELATIVITY_VENTURE',
                                            'RELATIVITY_OPERATION',
                                            'RELATIVITY_PACKAGE',
                                            'RELATIVITY_RADIUS',
                                            'RELATIVITY_VEHICLETYPE',
                                            'RELATIVITY_ILF',
                                            'RELATIVITY_FLEET',
                                            'RELATIVITY_DRIVER',
                                            'RELATIVITY_TERRITORY',
                                            'RERATED_PREMIUM_MODDED',
                                            'VEHICLE_COUNT',
                                            'RERATED_EARNED_PREMIUM')

ft_vehicle_coverage = fs.read_table(name='ess_feature_store.coverage')
ft_vehicle_coverage = (ft_vehicle_coverage.select('POLICYID',
                                                'POLICY_EFF_DT',
                                                'EXPOSURE_NO',
                                                'LIMIT_CSL',
                                                'LIMIT_BI_PERSON',
                                                'LIMIT_BI_OCCURRENCE',
                                                'LIMIT_PD',
                                                'DEDUCTIBLE_LIAB_TYPE',
                                                'DEDUCTIBLE_LIAB_AMT')
                       .withColumnRenamed('EXPOSURE_NO', 'VEHICLE_NO')
                      )


ft_vehicle_financials = fs.read_table(name='ess_feature_store.vehicle_financials')
ft_vehicle_financials = ft_vehicle_financials.select('POLICYID',
                                                    'POLICY_EFF_DT',
                                                    'VEHICLE_NO',
                                                    'PREM_WRITTEN_TRANS',
                                                    'PREM_EARNED_TRANS',
                                                    'PREM_WRITTEN_EFF',
                                                    'PREM_EARNED_EFF',
                                                    'OLEP_TRANS',
                                                    'OLEP_EFF')

ft_exposure_claims = fs.read_table(name='ess_feature_store.exposure_claims')
ft_exposure_claims = ft_exposure_claims.select('POLICYID',
                                              'POLICY_EFF_DT',
                                              'VEHICLE_NO',
                                              'COVERAGE_GROUP',
                                              'INDEMNITY_PAID',
                                              'INDEMNITY_INCURRED',
                                              'EXPENSE_PAID',
                                              'EXPENSE_INCURRED',
                                              'LARGE_LOSSES_100K',
                                              'LARGE_LOSSES_250K',
                                              'LARGE_LOSSES_500K',
                                              'LARGE_LOSSES_1M',
                                              'LATE_REPORTED_CLAIMS',
                                              'CLAIM_COUNT_PAID',
                                              'CLAIM_COUNT_NOT_PAID')

ft_cab = fs.read_table(name='ess_feature_store.vehicle_cab')
ft_cab = ft_cab.select('DOT_NUMBER',
                      'CRASH_SCORE',
                      'BASIC_ALERTS',
                      'DRIVER_INSPECTIONS',
                      'DRIVER_OUT_OF_SERVICE',
                      'DRIVER_OOS_RATIO',
                      'VEHICLE_INSPECTIONS',
                      'VEHICLE_OUT_OF_SERVICE',
                      'VEHICLE_OOS_RATIO',
                      'DRIVING_WITHOUT_CDL',
                      'DRIVING_WO_CDL_RATIO',
                      'MCS150_UNIT_COUNT',
                      'MILEAGE_POWER_UNIT_RATIO',
                      'DRIVER_POWER_UNIT_RATIO',
                      'HOT_ZONE_INsPECTIONS',
                      'HOT_ZONE_RATIO',
                      'PREVIOUS_INSURERS',
                      'LENGTH_WITH_PRIOR',
                      'CRASHES_SEASON_1',
                      'CRASHES_SEASON_2',
                      'CRASHES_SEASON_3',
                      'CRASHES_SEASON_4')


In [0]:
rubix5_eda = (ft_policy.join(ft_vehicle_policy, ['POLICYID', 'POLICY_EFF_DT'], how='left')
              # Vehicle Data
             .join(ft_vehicle_exposure, ['POLICYID', 'POLICY_EFF_DT'], how='left')
              # Pricing Data 
              .join(ft_vehicle_pricing, ['POLICYID', 'POLICY_EFF_DT'], how='left')
              # Rerating data
              .join(ft_vehicle_rating, ['POLICYID', 'POLICY_EFF_DT', 'VEHICLE_NO'], how='left')
              # Coverage data
              .join(ft_vehicle_coverage, ['POLICYID', 'POLICY_EFF_DT', 'VEHICLE_NO'], how='left')
              # Vehicle financials
              .join(ft_vehicle_financials, ['POLICYID', 'POLICY_EFF_DT', 'VEHICLE_NO'], how='left')
              .join(ft_exposure_claims, ['POLICYID', 'POLICY_EFF_DT', 'VEHICLE_NO'], how='left')
              #.join(ft_cab, ['DOT_NUMBER'], how='left')
              .filter(col('POLICY_EFF_DT')>='2015-01-01')
              .dropna(subset='VEHICLE_NO')
              .dropDuplicates(['POLICYID', 'POLICY_EFF_DT', 'VEHICLE_NO'])
              .withColumn("DOT_NUMBER", when(col("DOT_NUMBER").isNull(), lit("Unknown")).otherwise(col("DOT_NUMBER")))
              .withColumn("COVERAGE_GROUP", lit('CSL'))
              .fillna(0, subset=['INDEMNITY_PAID', 'INDEMNITY_INCURRED', 'EXPENSE_PAID', 'EXPENSE_INCURRED', 'LARGE_LOSSES_100K', 'LARGE_LOSSES_250K', 'LARGE_LOSSES_500K', 'LARGE_LOSSES_1M', 'LATE_REPORTED_CLAIMS', 'CLAIM_COUNT_PAID', 'CLAIM_COUNT_NOT_PAID'])
             )

rubix5_eda.count()

Out[5]: 238036

In [0]:
display(rubix5_eda)

POLICYID,POLICY_EFF_DT,VEHICLE_NO,POLICY_EXP_DT,POLICYSTATUS,POLICYISSUETYPE,COMPANY,POLICYSTATE,STATE_ADMITTED,NEW_VENTURE,PKGCOMBO,YRSINBUSCT,YRSOFINSCT,POLICY_SEGMENT,ZONE_GRPCD_POLICY,DOT_NUMBER,SCHEDULED_FLEET,VEHICLE_SEGMENT,VEHICLE_TERRITORY,VEHICLE_STATE,MODEL_YEAR,EQUIPMENT_AGE,VIN,RADIUS_GROUP,CLASS_CD,ZONE_GRPCD_VEHICLE,VEHICLE_ZIPCD,CLASS_PRIMARY,CLASS_SECONDARY,VEHICLE_TYPE,TRANSFORM_TERRITORY,TERRITORY_GRP,OPERATION_EXPOSURE_VEHICLE,EARNED_EXP_TRANS,EARNED_EXPOSURES,AVG_DRIVER_SURCHARGE,LIAB_SCHMOD,LIAB_EXPERIMOD,RERATED_PREMIUM,BASE_RATE,RELATIVITY_EQUIPMENT,RELATIVITY_VENTURE,RELATIVITY_OPERATION,RELATIVITY_PACKAGE,RELATIVITY_RADIUS,RELATIVITY_VEHICLETYPE,RELATIVITY_ILF,RELATIVITY_FLEET,RELATIVITY_DRIVER,RELATIVITY_TERRITORY,RERATED_PREMIUM_MODDED,VEHICLE_COUNT,RERATED_EARNED_PREMIUM,LIMIT_CSL,LIMIT_BI_PERSON,LIMIT_BI_OCCURRENCE,LIMIT_PD,DEDUCTIBLE_LIAB_TYPE,DEDUCTIBLE_LIAB_AMT,PREM_WRITTEN_TRANS,PREM_EARNED_TRANS,PREM_WRITTEN_EFF,PREM_EARNED_EFF,OLEP_TRANS,OLEP_EFF,COVERAGE_GROUP,INDEMNITY_PAID,INDEMNITY_INCURRED,EXPENSE_PAID,EXPENSE_INCURRED,LARGE_LOSSES_100K,LARGE_LOSSES_250K,LARGE_LOSSES_500K,LARGE_LOSSES_1M,LATE_REPORTED_CLAIMS,CLAIM_COUNT_PAID,CLAIM_COUNT_NOT_PAID
CAF0000007,2019-04-15,058,2020-04-15,Renewed,Renewal,FREEDOM SPECIALTY,GA,A,OTHER,AUTO LIABILITY ONLY,22,null,Brokerage Trans,,978324,YES,Brokerage Trans,123,GA,2020,0-5,5PVNV8JT4L5S55594,LOCAL,334590,,30188,Heavy Commercial Local Non-fleet,Waste Disposal - All Other,Heavy Commercial Non Zone Truckers,123,GA02,FLATBED,0.2547945205479452,0.2547945205479452,1.0,1.0000000000,1.0000000000,null,null,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,null,59,null,1000000,null,null,null,,,1940.0,1939.999999999999,1940.0,1940.0,2882.679999999999,2882.67,CSL,0.00,0.00,0.00,0.00,0,0,0,0,0,0,0
CAF0000009,2019-05-31,014,2020-05-31,Cancelled,Renewal,FREEDOM SPECIALTY,FL,A,2ND YR NEW VENTURE,AUTO LIABILITY ONLY,1,null,Brokerage Trans,,3117978,YES,Brokerage Trans,105,FL,2019,0-5,1M2GR2GC7KM007440,LOCAL,404590,,32808,Extra Heavy Local Non-fleet,Waste Disposal - All Other,Extra Heavy Non Zone Truckers,105,FL02,HAZ MAT,0.0,0.7917808219178081,1.0,1.0000000000,1.0000000000,null,null,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,null,22,null,1000000,null,null,null,,,15733.0,15733.000000000004,15733.0,15732.999999999996,23377.860000000004,23377.89,CSL,0.00,0.00,0.00,0.00,0,0,0,0,0,0,0
CAF0000010,2019-06-01,005,2020-06-01,Renewed,New,FREEDOM SPECIALTY,IL,A,OTHER,AUTO LIABILITY ONLY,5,null,Brokerage Trans,,Unknown,YES,Brokerage Trans,132,IL,2012,6-15,3HAMMAAL8CL608372,LONG,335490,,61601,Heavy Commercial Intermediate Non-fleet,Specialized Delivery - All Other,Heavy Commercial Non Zone Truckers,132,IL03,HAZ MAT,0.33150684931506846,0.33150684931506846,1.0,1.0000000000,1.0000000000,null,null,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,null,23,null,1000000,null,null,null,,,3882.0,3882.0000000000014,3882.0,3882.0,5768.310000000001,5768.3099999999995,CSL,0.00,0.00,0.00,0.00,0,0,0,0,0,0,0
CAF0000019,2020-01-06,052,2021-01-06,Did Not Renew,New,FREEDOM SPECIALTY,FL,A,OTHER,AUTO LIABILITY ONLY,5,null,Brokerage Trans,,Unknown,YES,Brokerage Trans,157,FL,2019,0-5,2GCRCNEC1K1179283,LOCAL,014990,,34994,Light Service Local Non-fleet,Not Otherwise Specified - All Other,Service Non Zone Truckers,157,FL03,HAZ MAT,1.0027397260273971,1.0027397260273971,1.0,1.0000000000,1.0000000000,null,null,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,null,71,null,1000000,null,null,null,,,2674.0,2673.9999999999845,2674.0,2674.0,4013.439999999985,4013.44,CSL,0.00,0.00,0.00,0.00,0,0,0,0,0,0,0
CAF0000019,2020-01-06,058,2021-01-06,Did Not Renew,New,FREEDOM SPECIALTY,FL,A,OTHER,AUTO LIABILITY ONLY,5,null,Brokerage Trans,,Unknown,YES,Brokerage Trans,157,FL,2019,0-5,3GCPYAEH8KG213546,LOCAL,014990,,34994,Light Service Local Non-fleet,Not Otherwise Specified - All Other,Service Non Zone Truckers,157,FL03,HAZ MAT,0.9205479452054793,0.9205479452054793,1.0,1.0000000000,1.0000000000,null,null,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,null,71,null

In [0]:
rubix5_eda.write.mode('overwrite').saveAsTable('es.rubix5_csl_allfleet')